In [19]:
#!pip3 install transformers datasets
import transformers
from datasets import load_dataset, load_metric



In [20]:
#from google.colab import drive
#drive.mount('/content/drive')

We'll use Weights and Biases for dislaying logging information during training. You will need to sign up for the service at `wanb.ai`. This will give you an access token which you can paste when running `wanb login` 

In [140]:
#!pip3 install wandb
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [141]:
#from datasets import load_dataset

#data = load_dataset('csv', data_files='./drive/MyDrive/ColabNotebooks/finer/GPT-3_data_split_into_128_word_paragraphs.csv', delimiter=',', column_names=["job", "filtered_skills"])


In [142]:
#!pip install dataset

In [143]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset from the CSV file
data = pd.read_csv("./GPT-3_data_split_into_128_word_paragraphs.csv")

In [144]:
data1 = data.dropna()
data1 = data1[['job_segment', 'skills']]
data1, test_data = train_test_split(data1, test_size=0.05, random_state=42)
data1.head()

,job_segment,skills
5859,Boy have we got some fun things for you…You ha...,"MS, BS"
2391,"Exciting Remote Cloud Azure Security Engineer,...",Security Engineering
6527,"To be successful in this role, you need a stro...","software development, machine learning"
454,The company joins with its employees to take a...,"ExaCC, database, Cloud implementations, Oracle..."
4792,Synergistic IT is a full-service staffing and ...,IT


In [145]:
data1['job_segment'] = data1['job_segment'].replace('[^A-Za-z0-9\s]+', '', regex=True)

In [146]:
data1.head()

,job_segment,skills
5859,Boy have we got some fun things for youYou hav...,"MS, BS"
2391,Exciting Remote Cloud Azure Security Engineer ...,Security Engineering
6527,To be successful in this role you need a stron...,"software development, machine learning"
454,The company joins with its employees to take a...,"ExaCC, database, Cloud implementations, Oracle..."
4792,Synergistic IT is a fullservice staffing and p...,IT


In [147]:
# Split the dataset into training and development sets
train_data, dev_data = train_test_split(data1, test_size=0.10, random_state=42)

# Convert the training and development sets into HuggingFace dataset format
from datasets import Dataset

train = Dataset.from_pandas(train_data)
dev = Dataset.from_pandas(dev_data)

Let's examine the training and development data:

In [148]:
print(train)
print(dev)

Dataset({
    features: ['job_segment', 'skills', '__index_level_0__'],
    num_rows: 3657
})
Dataset({
    features: ['job_segment', 'skills', '__index_level_0__'],
    num_rows: 407
})


And, print the first training example:

In [149]:
print(train[1])

{'job_segment': 'If you have what it takes then look no furtherRequirementsTop Requirements 4 years with strong experience with SQL Excellent written and interpersonal communication skills Strong understanding of IT security and data privacy principles including experience working under at least one of the following HIPPA FEPRA PCI or SOX Expert knowledge with database structures principles and practices or of datadriven application designFOR IMMEDIATE CONSIDERATION PLEASE MESSAGE ALI SCOTT ON LINKEDIN WITH YOUR UPDATED RESUMETechnology Doesnt Change the World People DoRobert Half is the worlds first and largest specialized talent solutions firm that connects highly qualified job seekers to opportunities at great companies We offer contract temporary and permanent placement solutions for finance and accounting technology marketing and creative legal and administrative and customer support rolesRobert Half puts you in the best position to succeed by advocating on your behalf and promoti

In [31]:
train_data.shape

(3657, 2)

We'll now start to process our data. We will need to use `nltk.word_tokenize`, so we need to load the `nltk` module. Additionally, we'll load the Flan T5 tokenizer.

In [34]:
#!pip install nltk
import nltk
nltk.download('punkt')
import string
from transformers import AutoTokenizer

model_checkpoint = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
PREFIX = "what are the skills in :"
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 64

def preprocess_data(examples):
    inputs = [PREFIX + " " + text for text in examples["job_segment"]]
    
    model_inputs = tokenizer(inputs, 
                             max_length=MAX_INPUT_LENGTH, 
                             truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["skills"], 
                           max_length=MAX_TARGET_LENGTH, 
                           truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

We'll then apply the tokenization function to our datasets

In [36]:
tokenized_train = train.map(preprocess_data, batched=True)
tokenized_dev = dev.map(preprocess_data, batched=True)

Map:   0%|          | 0/3657 [00:00<?, ? examples/s]

/opt/conda/envs/pytorch/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/407 [00:00<?, ? examples/s]

Let's print the first tokenized training example:

In [37]:
tokenized_train[0]

{'job_segment': ' Demonstrate advanced working knowledge and experience with machine learning algorithms and populationbased metaheuristic optimization methodsGenerate innovative ideas establish new research directions and shape and execute on technical projectsMaintain stateoftheart knowledge and contribute to technical discussions and reviews as an expert in related areas of responsibilityApply theoretical knowledge to solve industrial problemsApply engineering knowledge in developing datadriven algorithms for anomaly detection failure prediction and optimizationCollaborate with field and product engineers to identify key health monitoring parameters of a systemProcess large multivariate data sets collected from equipment operations manufacturing tests and diagnostic routinesCommunicate ideas plans and results effectively via oral and written reportsBasic QualificationsMasters or PhD degree in science or engineering field requiredMinimum 3 years of direct experience MA Degree or 1 ye

Next we will import classes which are used for fine-tuning of seq2seq models like Flan-T5.

* `AutoModelForSeq2SeqLM` loads the model itself,
* `DataCollatorForSeq2Seq` takes care of data batching,
* `Seq2SeqTrainingArguments` sets all hyperparameters for training, and
* `Seq2SeqTrainer` trains the model

In [38]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

We'll then set the hyperparameters for model training:

```
model_dir -- where to save model checkpoints
evaluation_strategy="steps" -- evaluate every N steps
eval_steps=100 -- where N = 100
logging_strategy="steps" -- write information about training loss  every N steps
logging_steps=1 -- where N = 1
save_strategy="steps" -- save model every N steps
save_steps=100 -- where N = 100
learning_rate=4e-5 -- initial learning rate for training
per_device_train_batch_size=batch_size -- training batch size
per_device_eval_batch_size=batch_size -- evaluation batch size
weight_decay=0.01 -- hyperparameter for weight decay during training
save_total_limit=3 -- save a maximum of 3 models
num_train_epochs=1 -- number of training epochs
predict_with_generate=True -- generate the actual output during evaluation 
```

In [39]:
batch_size = 2
model_name = "flan-t5-finer"
model_dir = f"./models/{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=4e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True)

Let's define a data collator which batches the training and development data for us:

In [40]:
data_collator = DataCollatorForSeq2Seq(tokenizer, padding=True)

We'll use BLEU score to evaluate progress during training because our outputs are token sequences and even partial matches can be valuable

In [41]:
#  !pip3 install sacrebleu

In [42]:
# This is potentially required because of some obscure colab bug
import locale
locale.getpreferredencoding = lambda: "UTF-8"

#!pip3 install sacrebleu
metric = load_metric("sacrebleu")

/tmp/ipykernel_37796/409792425.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


We'll now define a function to compute BLEU score based on model predictions and gold standard outputs.

# Compute Metrics

In [51]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # The labels are padded to equal length using a special symbol -100.
    # This is not the regular <PAD> symbol for the tokenizer so
    # we'll have to replace -100 in the labels before decoding.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # For BLEU, we'll need to split the outputs into sequences
    # of word tokens. We'll just use NLTK word_tokenize.
    decoded_preds = [nltk.word_tokenize(pred.strip())
                      for pred in decoded_preds]
    decoded_labels = [[nltk.word_tokenize(label.strip())]
                      for label in decoded_labels]
    
    # Compute BLEU scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Return BLEU scores
    return result 


def compute_metrics1(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels before decoding
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds_tokens = [nltk.word_tokenize(pred.strip()) for pred in decoded_preds]
    decoded_labels_tokens = [[nltk.word_tokenize(label.strip())] for label in decoded_labels]

    # Compute BLEU scores
    bleu_result = metric.compute(predictions=decoded_preds_tokens, references=decoded_labels_tokens)

    # Compute the "accuracy" based on exact string matches
    total_samples = len(decoded_preds)
    correct_samples = sum([1 if pred.strip() == label.strip() else 0 for pred, label in zip(decoded_preds, decoded_labels)])
    accuracy = correct_samples / total_samples

    # Return BLEU scores and accuracy
    return {"bleu": bleu_result, "accuracy": accuracy}


In [52]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [53]:
model = model_init()
text = """SummaryThe Database Developer is part of the CEMCO development team whose responsibilities include managing and maintaining the enterprise data warehouse.Strong development skills in C# and .Net, including Excel VSTO.The ideal candidate should have a deep understanding of database management systems, as well as the ability to write complex code in C# and .Net to build, maintain and optimize databases.Effective at communicating with users to define requirements and design, solutions based on those requirements.This position is also responsible for creating tools that provide insight to various departments, including Finance, Accounting, and Customer Service to build data visualizations and dashboards.Essential Duties And ResponsibilitiesDesign, install, configure, and maintain database management systems.Monitor database performance and provide optimization.Develop, implement, and maintain backup and recovery procedures."""
inputs = ["what are the skills in : " + text]

print("INPUT:", inputs)
inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=32, do_sample=True, min_length=1, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print("OUTPUT:", decoded_output)

INPUT: ['what are the skills in : SummaryThe Database Developer is part of the CEMCO development team whose responsibilities include managing and maintaining the enterprise data warehouse.Strong development skills in C# and .Net, including Excel VSTO.The ideal candidate should have a deep understanding of database management systems, as well as the ability to write complex code in C# and .Net to build, maintain and optimize databases.Effective at communicating with users to define requirements and design, solutions based on those requirements.This position is also responsible for creating tools that provide insight to various departments, including Finance, Accounting, and Customer Service to build data visualizations and dashboards.Essential Duties And ResponsibilitiesDesign, install, configure, and maintain database management systems.Monitor database performance and provide optimization.Develop, implement, and maintain backup and recovery procedures.']
OUTPUT: Finance and IT departm

In [54]:
trainer.train()

Step,Training Loss,Validation Loss,Score,Counts,Totals,Precisions,Bp,Sys Len,Ref Len
1000,2.674500,2.139409,34.178518,"[2670, 1631, 1257, 935]","[3947, 3540, 3133, 2726]","[67.64631365594123, 46.07344632768361, 40.12128949888286, 34.2993396918562]",0.751042,3947,5077
2000,0.608400,2.027874,29.866704,"[2452, 1448, 1107, 817]","[3482, 3075, 2668, 2261]","[70.4192992533027, 47.08943089430894, 41.49175412293853, 36.134453781512605]",0.632503,3482,5077
3000,1.883800,2.001194,33.919467,"[2622, 1607, 1245, 944]","[3980, 3573, 3166, 2759]","[65.87939698492463, 44.976210467394345, 39.32406822488945, 34.21529539688293]",0.759095,3980,5077


TrainOutput(global_step=3658, training_loss=2.2442769999343426, metrics={'train_runtime': 495.7602, 'train_samples_per_second': 14.753, 'train_steps_per_second': 7.379, 'total_flos': 338898721849344.0, 'train_loss': 2.2442769999343426, 'epoch': 2.0})

After training, we can load the best checkpoint model from Google Drive:

In [126]:
model_name = "flan-t5-finer"
model_dir = f"./models/{model_name}/checkpoint-2000"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = MAX_INPUT_LENGTH

We can then test the model on the same input as before. This time, the output seems to be correct: `Ville Oksanen` 

In [127]:
text = """SummaryThe Database Developer is part of the CEMCO development team whose responsibilities include managing and maintaining the enterprise data warehouse.Strong development skills in C# and .Net, including Excel VSTO.The ideal candidate should have a deep understanding of database management systems, as well as the ability to write complex code in C# and .Net to build, maintain and optimize databases.Effective at communicating with users to define requirements and design, solutions based on those requirements.This position is also responsible for creating tools that provide insight to various departments, including Finance, Accounting, and Customer Service to build data visualizations and dashboards.Essential Duties And ResponsibilitiesDesign, install, configure, and maintain database management systems.Monitor database performance and provide optimization.Develop, implement, and maintain backup and recovery procedures."""

inputs = ["what are the skills in : " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=32, do_sample=True, min_length=1, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print(decoded_output)

C#,.Net


In [128]:

text="""About The JobWe are seeking a Senior Full Stack Software Engineer to build and scale a variety of Web and Mobile Products for an intriguing range of clients.This is a unique opportunity to work alongside an experienced team that excels at building SaaS products.This position reports to a Development Team Lead.QualificationsEssential skills: Deep experience with Java / Spring Boot, C# / .NET, or React (with functional components)Helpful skills: Node.js, React Native, Angular, or Ruby on RailsThe role also requires a voracious appetite to learn and apply new skills.You should be skilled at building web-based software or mobile apps, and excel at learning new languages and frameworks."""
inputs = ["what are the skills in : " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=32, do_sample=True, min_length=1, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print(decoded_output)

Node.js, React, Angular


In [129]:

text="""What Do You Need To Succeed?Must HaveMasters or PHD in Mathematics, Physics, Statistics, Computer Science or other quantitative fields.2+ years of experience in Data Science, Machine-Learning, Natural Language Processing or StatisticsOther Required Qualifications:Out of the box thinking about data science and data sourcesCreative approaches to using datasets to get insights into financial performance of companiesStrong in data profiling, cleaning, and insight generationAbility to perform complex data analysis on large volumes of data and present findings to stakeholders.Strong knowledge of design, development, and implementation experience utilizing data science technologies.Excellent analytical and problem solving skills.Expert technical documentation skills.Familiar with a range of data tools.Work experience with complex visualization of data.Expert working knowledge of Python and RKnowledgeable in SQLStrong coding abilities.Familiar with a Linux environment and shell scripting."""
inputs = ["what are the skills in : " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=32, do_sample=True, min_length=1, max_length=64)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
print(decoded_output)

Data Science, Machine-Learning


In [130]:
# new_column = []

# for index, row in data.iterrows():
#     text=row["job_segment"]
#     inputs = ["what are the skills in : " + text]
#     inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
#     output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=1, max_length=max_input_length)
#     decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]

#     new_column.append([i.strip() for i in decoded_output.split(",")])
#     if index == 100:
#         break

In [131]:
#data.loc[:100, 'predicted_skills'] = new_column

In [132]:
#pred = data.loc[:100]

In [133]:
#pred.head()

In [134]:
#pred['skills'] = pred['skills'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)


In [135]:
#pred

In [150]:
data_pred = test_data#pd.read_csv("./GPT-3_skills_filtered_data_test.csv")
new_column = []

data_pred.head()

,job_segment,skills
793,Knowledge of End-End Test Automation usingTest...,"React.js, React, CSS, JavaScript"
579,Our cloud-based platform incorporates advanced...,"AR, Augmented Reality, React, HCI"
4059,The Cloud Engineer will contribute to the desi...,"public cloud, Cloud Engineer, Cloud, Cloud Sec..."
1970,"', ""And as we work today to drive better care ...","machine learning, ResMed"
885,"In This Role, You’llSupport Sales team to asse...",Engineering


In [151]:

def process_skills(skills):
    if not isinstance(skills, str) or skills == "NaN":
        return []
    return [skill.strip() for skill in skills.split(',')]

data_pred['skills'] = data_pred['skills'].apply(process_skills)

In [152]:
data_pred

,job_segment,skills
793,Knowledge of End-End Test Automation usingTest...,"[React.js, React, CSS, JavaScript]"
579,Our cloud-based platform incorporates advanced...,"[AR, Augmented Reality, React, HCI]"
4059,The Cloud Engineer will contribute to the desi...,"[public cloud, Cloud Engineer, Cloud, Cloud Se..."
1970,"', ""And as we work today to drive better care ...","[machine learning, ResMed]"
885,"In This Role, You’llSupport Sales team to asse...",[Engineering]
...,...,...
5999,What you’ll do:Design and execute the querying...,[SQL]
4370,""", 'Must be detail oriented.Experience SQL Ser...",[SQL Server]
4773,We believe in a collective responsibility to c...,[ACLU]
4891,Maintains and develops security requirements b...,"[vulnerability scanning, secure, security]"


In [153]:
for index, row in data_pred.iterrows():
    text=row["job_segment"]
    inputs = ["what are the skills in : " + text]
    inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
    output = model.generate(**inputs, num_beams=32, do_sample=True, min_length=1, max_length=max_input_length)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]

    new_column.append([i.strip() for i in decoded_output.split(",")])

In [154]:
data_pred['predicted_skills'] = new_column

In [155]:
data_pred.head()

,job_segment,skills,predicted_skills
793,Knowledge of End-End Test Automation usingTest...,"[React.js, React, CSS, JavaScript]","[React.js, React.js]"
579,Our cloud-based platform incorporates advanced...,"[AR, Augmented Reality, React, HCI]","[Artificial Intelligence, Artificial Intellige..."
4059,The Cloud Engineer will contribute to the desi...,"[public cloud, Cloud Engineer, Cloud, Cloud Se...",[AWS]
1970,"', ""And as we work today to drive better care ...","[machine learning, ResMed]",[MatrixCare]
885,"In This Role, You’llSupport Sales team to asse...",[Engineering],[Roboflow]


In [156]:
data_pred1 = data_pred

In [157]:
data_pred1.head()

,job_segment,skills,predicted_skills
793,Knowledge of End-End Test Automation usingTest...,"[React.js, React, CSS, JavaScript]","[React.js, React.js]"
579,Our cloud-based platform incorporates advanced...,"[AR, Augmented Reality, React, HCI]","[Artificial Intelligence, Artificial Intellige..."
4059,The Cloud Engineer will contribute to the desi...,"[public cloud, Cloud Engineer, Cloud, Cloud Se...",[AWS]
1970,"', ""And as we work today to drive better care ...","[machine learning, ResMed]",[MatrixCare]
885,"In This Role, You’llSupport Sales team to asse...",[Engineering],[Roboflow]


In [158]:
from sklearn.metrics import precision_recall_fscore_support

# Convert the lists in the 'skills' column into sets
data_pred1['skills'] = data_pred1['skills'].apply(lambda x: set(x) if isinstance(x, list) else x)

# Convert the lists in the 'predicted_skills' column into sets
data_pred1['predicted_skills'] = data_pred1['predicted_skills'].apply(lambda x: set(x) if isinstance(x, list) else x)




In [159]:
data_pred1.head()

,job_segment,skills,predicted_skills
793,Knowledge of End-End Test Automation usingTest...,"{CSS, React.js, JavaScript, React}",{React.js}
579,Our cloud-based platform incorporates advanced...,"{AR, HCI, Augmented Reality, React}","{Artificial Intelligence, Artificial Intel}"
4059,The Cloud Engineer will contribute to the desi...,"{Python, Cloud, Cloud Engineer, Go, Cloud Secu...",{AWS}
1970,"', ""And as we work today to drive better care ...","{machine learning, ResMed}",{MatrixCare}
885,"In This Role, You’llSupport Sales team to asse...",{Engineering},{Roboflow}


In [160]:
from sklearn.metrics import precision_recall_fscore_support

# Convert the 'filtered_skills' and 'predicted_skills' columns to lists
y_true = data_pred1['skills'].tolist()
y_pred = data_pred1['predicted_skills'].tolist()




# Evaluate

In [1]:
#y_pred[2:5]

In [2]:
#y_true[2:5]

In [163]:
def evaluate(gold, pred):
    gold_count = 0
    pred_count = 0
    TP = 0
    for i,j in zip(gold, pred):
        TP += len(i.intersection(j))
        gold_count += len(i)
        pred_count += len(j)
        
    prec = TP / pred_count
    recall = TP / gold_count
    f1 = 2 * prec * recall / (prec + recall)
    return prec, recall, f1

evaluate(y_true, y_pred)


(0.39166666666666666, 0.29436325678496866, 0.33611442193087004)